In [ ]:
from subprocess import Popen

In [ ]:
expe_name='4-23'

CFs=[4000, 5000, 6000, 8000]

E0_distributed=True
Q10_distributed=True



In [ ]:

list_args0=['papermill', '-p', 'E0_distributed',  str(E0_distributed), 
       '-p', 'Q10_distributed', str(Q10_distributed), '-p', 'n_workers',
       str(len(CFs)+1), '-p', 'rank']

processes=[]
for i, CF in enumerate(CFs):
    list_args=list_args0+ [str(i+1), '-p',  'CF', str(CF), 'Fit\ data.ipynb', f'fitdata{CF}.ipynb']
    st_cmd=' '.join(list_args)
    print(st_cmd)
    #processes.append(Popen(st_cmd, shell=True))